# Required Imports

In [ ]:
import requests
import pandas as pd
import time

# Get Tide Data from NOAA

In [ ]:
Nikiski = 9455760

def getTideData():
    start_date = 19830611
    end_date = 20240809
    new_start = None
    new_end = None
    allData = pd.DataFrame()
    Time = []
    Tide_Value = []
    Type = []

    for year in range(1983,2025):
        for month in range(3,13,3):
            
            begin_cur_month = month
           
            if(month<10):
                cur_month = f"{0}{month}"

            new_start = f"{year}{cur_month}01"
            
            if(month == 12):
                    new_end = f"{year+1}0301"
            else:
                if(month <9):
                    new_end = f"{year}{0}{month+3}01"
                else:
                    new_end = f"{year}{month+3}01"

               
            response = requests.get(f"https://api.tidesandcurrents.noaa.gov/api
            /prod/datagetter?begin_date={new_start}&end_date={new_end}&station={Nikiski}&
            product=predictions&datum=MLLW&time_zone=lst_ldt&interval=hilo&units=english&
            application=DataAPI_Sample&format=json")

            if response.status_code == 200:
                Data = response.json()
               
                
                for Obs in Data["predictions"]:
                    Time.append(Obs["t"])
                    Tide_Value.append(Obs["v"])
                    if Obs["type"] == "L":
                        Type.append("Low")
                    else:
                        Type.append("High")

            else:
                print("Request Failed")
                print("Start",new_start)
                print("End",new_end)
            
            time.sleep(15)
            print(end_date)
    Data = pd.DataFrame({
                         "Time":Time,
                         "Tide_Value": Tide_Value,
                         "Type": Type
                         })
    
    Data.to_csv("TideData.csv")
            

# Get Water Temperature Data

In [ ]:
def getWaterTemp():

    new_start = None
    new_end = None
    allData = pd.DataFrame()
    Time = []
    Temp = []
    
    for year in range(1997,2025):
        for month in range(3,13,3):
            
            begin_cur_month = month
           
            if(month<10):
                cur_month = f"{0}{month}"

            new_start = f"{year}{cur_month}01"
            
            if(month == 12):
                    new_end = f"{year+1}0301"
            else:
                if(month <9):
                    new_end = f"{year}{0}{month+3}01"
                else:
                    new_end = f"{year}{month+3}01"

            response = requests.get(f"https://api.tidesandcurrents.noaa.gov/api/prod/datagetter?
            begin_date={new_start}&end_date={new_end}&station={Nikiski}&product=water_temperature&
            time_zone=lst_ldt&units=english&interval=h&format=json")

            if response.status_code == 200:
                Data = response.json()
               
                if("error" in Data.keys()):
                    continue 
                
                for Obs in Data["data"]:
                    Time.append(Obs["t"])
                    Temp.append(Obs["v"])

                    if Obs["v"] != "":
                        if float(Obs["v"]) < 20: 
                            print("Invalid Ultra Low Temp", Obs["v"])

            else:
                print("Request Failed")
                print("Start",new_start)
                print("End",new_end)
            
            time.sleep(15)
            print(new_end)

    Data = pd.DataFrame({
                         "Time":Time,
                         "Type": Temp
                         })
    
    Data.to_csv("TempData.csv")


# Run Data Collection Scripts

In [ ]:
if __name__ == "__main__":
    getTideData()
    getWaterTemp()

#### The historical fish data was conveniantly available to download in excel format through the Alaska Department of Fish and Game 
 
#### All of the data was then combined using Excel and associated tools like Power Query to get the MergedData.csv file